In [1]:
import pandas as pd
from metasynth import MetaDataset
import wget
from pathlib import Path
import json

In [2]:
dtypes = {
    "Survived": "category",
    "Pclass": "category",
    "Name": "string",
    "Sex": "category",
    "SibSp": "category",
    "Parch": "category",
    "Ticket": "string",
    "Cabin": "string",
    "Embarked": "category"
}

In [3]:
titanic_fp = Path("titanic.csv")
if not titanic_fp.is_file():
    wget.download("https://raw.githubusercontent.com/pandas-dev/pandas/main/doc/data/titanic.csv")

In [4]:
df = pd.read_csv(titanic_fp, dtype=dtypes)

In [5]:
dataset = MetaDataset.from_dataframe(df)

In [6]:
dataset.to_json("test.json")

In [7]:
print(dataset)

# Rows: 891
# Columns: 12

{'name': 'PassengerId', 'type': 'IntVar', 'dtype': 'int64', 'prop_missing': 0.0, 'distribution': "{'name': 'DiscreteUniformDistribution', 'parameters': {'low': 1, 'high': 892}}"}

{'name': 'Survived', 'type': 'CategoricalVar', 'dtype': 'category', 'prop_missing': 0.0, 'distribution': "{'name': 'CatFreqDistribution', 'parameters': {'cat_freq': {'0': 549, '1': 342}}}"}

{'name': 'Pclass', 'type': 'CategoricalVar', 'dtype': 'category', 'prop_missing': 0.0, 'distribution': "{'name': 'CatFreqDistribution', 'parameters': {'cat_freq': {'1': 216, '2': 184, '3': 491}}}"}

{'name': 'Name', 'type': 'StringVar', 'dtype': 'string', 'prop_missing': 0.0, 'distribution': '[A-Z]{0,1}[a-z]{0,12}[KRG,NM\'C-E D][IBJYoaLC PdGMvyDeFVHWS][A-Z]{0,1}[a-z]{0,10}[ .,][CM h][A-Z]{0,1}[a-z]{0,10}[,/ .(][A-Z]{0,1}[a-z]{0,10}[" )(][A-Z]{0,2}[a-z]{0,10}[" ).(][A-Z]{0,1}[a-z]{0,10}[" ).(L][A-Z]{0,1}[a-z]{0,10}[" )(][A-Z]{0,1}[a-z]{0,9}["M )][A-Z]{0,1}[a-z]{0,12}[ )"][A-Z]{0,1}[a-z]{0,7}["- )

In [8]:
print(MetaDataset.from_json("test.json"))

# Rows: 891
# Columns: 12

{'name': 'PassengerId', 'type': 'IntVar', 'dtype': 'int64', 'prop_missing': 0.0, 'distribution': "{'name': 'DiscreteUniformDistribution', 'parameters': {'low': 1, 'high': 892}}"}

{'name': 'Survived', 'type': 'CategoricalVar', 'dtype': 'category', 'prop_missing': 0.0, 'distribution': "{'name': 'CatFreqDistribution', 'parameters': {'cat_freq': {'0': 549, '1': 342}}}"}

{'name': 'Pclass', 'type': 'CategoricalVar', 'dtype': 'category', 'prop_missing': 0.0, 'distribution': "{'name': 'CatFreqDistribution', 'parameters': {'cat_freq': {'1': 216, '2': 184, '3': 491}}}"}

{'name': 'Name', 'type': 'StringVar', 'dtype': 'string', 'prop_missing': 0.0, 'distribution': '[A-Z]{0,1}[a-z]{0,12}[KRG,NM\'C-E D][IBJYoaLC PdGMvyDeFVHWS][A-Z]{0,1}[a-z]{0,10}[ .,][CM h][A-Z]{0,1}[a-z]{0,10}[,/ .(][A-Z]{0,1}[a-z]{0,10}[" )(][A-Z]{0,2}[a-z]{0,10}[" ).(][A-Z]{0,1}[a-z]{0,10}[" ).(L][A-Z]{0,1}[a-z]{0,10}[" )(][A-Z]{0,1}[a-z]{0,9}["M )][A-Z]{0,1}[a-z]{0,12}[ )"][A-Z]{0,1}[a-z]{0,7}["- )